In [1]:
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D,BatchNormalization,Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [2]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
img_gen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rescale=1./255)

In [4]:
train_gen = img_gen.flow_from_directory('../../../data_set/模型訓練資料/format_train/',target_size=(96,96),batch_size=32)

Found 68804 images belonging to 800 classes.


In [5]:
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus),
                  "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)


solve_cudnn_error()

1 Physical GPUs, 1 Logical GPUs


In [12]:
input_shape = (96,96,3)

convnet = Sequential()
convnet.add(
    Conv2D(64, (10, 10),
           activation='relu',
           input_shape=input_shape,
           kernel_initializer=initializers.RandomNormal(stddev=0.01),
           kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128, (7, 7), activation='relu',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256, (7, 7), activation='relu',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(512, (4, 4), activation='relu',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(BatchNormalization())
convnet.add(Dropout(0.4))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(1024, (2, 2), activation='relu',padding='same',
                   kernel_initializer=initializers.RandomNormal(stddev=0.01),
                   kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(Dense(1024, activation="relu", kernel_regularizer=l2(1e-3)))
convnet.add(Dropout(0.4))
convnet.add(Dense(800, activation="softmax"))

convnet.compile(optimizer= Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
convnet.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 87, 87, 64)        19264     
_________________________________________________________________
batch_normalization_17 (Batc (None, 87, 87, 64)        256       
_________________________________________________________________
dropout_29 (Dropout)         (None, 87, 87, 64)        0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 37, 37, 128)       401536    
_________________________________________________________________
dropout_30 (Dropout)         (None, 37, 37, 128)       0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 18, 18, 128)      

In [ ]:
convnet.fit(train_gen,steps_per_epoch=train_gen.samples/32,
           epochs=10,verbose=True)

C:\Users\USER\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\USER\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10
2150/2150 [==============================] - 346s 161ms/step - loss: 3.1666 - accuracy: 0.4608
Epoch 2/10
2150/2150 [==============================] - 571s 265ms/step - loss: 3.1378 - accuracy: 0.4656
Epoch 3/10
2150/2150 [==============================] - 506s 235ms/step - loss: 3.1220 - accuracy: 0.4716
Epoch 4/10
2150/2150 [==============================] - 495s 230ms/step - loss: 3.1034 - accuracy: 0.4757
Epoch 5/10
2150/2150 [==============================] - 509s 237ms/step - loss: 3.0809 - accuracy: 0.4822
Epoch 6/10
2150/2150 [==============================] - 557s 259ms/step - loss: 3.0731 - accuracy: 0.4824
Epoch 7/10
2150/2150 [==============================] - 592s 275ms/step - loss: 3.0686 - accuracy: 0.4832
Epoch 8/10
2150/2150 [==============================] - 568s 264ms/step - loss: 3.0372 - accuracy: 0.4903
Epoch 9/10
2150/2150 [==============================] - 618s 288ms/step - loss: 3.0289 - accuracy: 0.4931
Epoch 10/10
1964/2150 [=======================

In [14]:
convnet.save('../../../data_set/模型訓練資料/save/original_cnn.h5')